In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import nltk
import torchtext
from konlpy.tag import Kkma

tagger = Kkma()
import gensim

Using TensorFlow backend.


http://anie.me/On-Torchtext/

In [9]:
torchtext.vocab.pretrained_aliases

{'charngram.100d': <function torchtext.vocab.<lambda>>,
 'fasttext.en.300d': <function torchtext.vocab.<lambda>>,
 'fasttext.simple.300d': <function torchtext.vocab.<lambda>>,
 'glove.42B.300d': <function torchtext.vocab.<lambda>>,
 'glove.6B.100d': <function torchtext.vocab.<lambda>>,
 'glove.6B.200d': <function torchtext.vocab.<lambda>>,
 'glove.6B.300d': <function torchtext.vocab.<lambda>>,
 'glove.6B.50d': <function torchtext.vocab.<lambda>>,
 'glove.840B.300d': <function torchtext.vocab.<lambda>>,
 'glove.twitter.27B.100d': <function torchtext.vocab.<lambda>>,
 'glove.twitter.27B.200d': <function torchtext.vocab.<lambda>>,
 'glove.twitter.27B.25d': <function torchtext.vocab.<lambda>>,
 'glove.twitter.27B.50d': <function torchtext.vocab.<lambda>>}

In [ ]:
# glove = torchtext.vocab.GloVe('42B',300)

## Gensim으로 빠르게 Word vector 훈련시키기 

In [1]:
corpus = open('data/corpus.txt','r',encoding="utf-8").readlines()
corpus = [c[:-1] for c in corpus]

In [3]:
tokenized = [tagger.morphs(c) for c in corpus]

In [16]:
model = gensim.models.Word2Vec(tokenized, size=15, window=5, min_count=2, workers=4)

In [17]:
model.most_similar("토치")

[('하', 0.9874316453933716),
 ('을', 0.985340416431427),
 ('이', 0.985215425491333),
 ('는', 0.9851880669593811),
 ('글', 0.9819232821464539),
 ('다', 0.9816867113113403),
 ('고', 0.9768091440200806),
 ('가', 0.9768056869506836),
 ('의', 0.975877046585083),
 ('파이', 0.9756559729576111)]

In [21]:
len(model.wv.index2word)

344

In [31]:
model.wv.save_word2vec_format("data/word_vector_sample.bin",binary=True) # 저장

## KeyedVectors -> Numpy 

In [34]:
pretrained_vectors_model = gensim.models.KeyedVectors.load_word2vec_format("data/word_vector_sample.bin",binary=True)

In [35]:
pretrained_vectors_model['토치']

array([-0.03937732, -0.01539551,  0.11729443,  0.07238159,  0.11705646,
        0.0947059 ,  0.07662146,  0.19524413, -0.12553942, -0.09215615,
       -0.00106716,  0.05587648,  0.07590903,  0.01543585,  0.17271918],
      dtype=float32)

In [37]:
vocab = list(pretrained_vectors_model.vocab.keys()) # Word2Vec에서 사용한 vocab 

In [39]:
pretrained_vectors=[]
for vo in vocab:
    pretrained_vectors.append(pretrained_vectors_model[vo])
    
pretrained_vectors = np.vstack(pretrained_vectors)

In [40]:
pretrained_vectors.shape # 15차원의 벡터가 344개(vocab수)

(344, 15)

## Init Embedding matrix 

In [45]:
class MyModel(nn.Module):
    def __init__(self,vocab_size,embed_size):
        super(MyModel,self).__init__()
        
        self.embed = nn.Embedding(vocab_size,embed_size)
        
    def init_embed(self,pretrained_vectors):
        self.embed.weight.data = torch.from_numpy(pretrained_vectors).float()
    
    def forward(self,inputs):
        return self.embed(inputs)

In [46]:
model = MyModel(len(vocab),15)

In [47]:
model.embed.weight

Parameter containing:
-0.0249  0.9056 -0.1130  ...   0.7458  1.2223 -0.2556
-1.0357 -0.8036  0.9044  ...  -1.0589  0.8175 -1.3345
-0.5411  0.4340  1.0868  ...   2.0499 -0.1602 -1.7726
          ...             ⋱             ...          
-1.7266  1.3220 -0.0409  ...   1.1238  1.0680  0.9796
 0.2594 -0.5651 -0.4928  ...  -0.0558 -0.7087  0.0770
-0.3756 -0.7101  0.3688  ...   0.3938  0.3859 -2.1166
[torch.FloatTensor of size 344x15]

In [48]:
model.init_embed(pretrained_vectors)

In [49]:
model.embed.weight

Parameter containing:
-0.0194 -0.0021  0.0172  ...  -0.0183 -0.0026  0.0283
 0.0007  0.0117  0.0269  ...   0.0256  0.0006  0.0520
 0.0109  0.0050 -0.0078  ...  -0.0034 -0.0232  0.0173
          ...             ⋱             ...          
-0.0298 -0.0242  0.0299  ...  -0.0015 -0.0050  0.0262
 0.0068 -0.0312  0.0946  ...   0.0914  0.0560  0.1382
-0.0262  0.0134  0.0052  ...  -0.0165 -0.0196  0.0271
[torch.FloatTensor of size 344x15]

## TODO 

- 다음의 코퍼스를 Gensim을 이용해서 Word Vector를 학습시킨 후(몇 차원으로 할지, min_count 몇으로 할지는 자율)
- 하나의 임베딩 매트릭스를 가진 파이토치 모델의 파라미터로 로드하라
- (텐서보드에 임베딩된 단어들을 시각화하라)

In [14]:
corpus = nltk.corpus.gutenberg.sents('melville-moby_dick.txt')[:500]

In [15]:
corpus[:3] # tokenized sentence

[['[', 'Moby', 'Dick', 'by', 'Herman', 'Melville', '1851', ']'],
 ['ETYMOLOGY', '.'],
 ['(',
  'Supplied',
  'by',
  'a',
  'Late',
  'Consumptive',
  'Usher',
  'to',
  'a',
  'Grammar',
  'School',
  ')']]